In [ ]:
import cv2
import numpy as np
import mediapipe as mp
from tensorflow.keras.models import load_model
import time

In [ ]:
# Load the trained model
model = load_model('sign_language_model.keras')

In [ ]:
# Define the labels for each gesture
labels = ["hello", "iloveyou", "no", "thanks", "yes"]

In [ ]:
# Initialize MediaPipe hands module
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

In [ ]:
# Initialize webcam
video_capture = cv2.VideoCapture(0)

# Real-time prediction
with mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.5) as hands:
    while video_capture.isOpened():
        ret, frame = video_capture.read()
        
        if not ret:
            print("Error: Failed to capture frame.")
            break
        
        # Flip the frame for a more natural experience
        frame = cv2.flip(frame, 1)
        
        # Convert the frame to RGB as MediaPipe requires RGB images
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Process the frame and find hands
        results = hands.process(rgb_frame)
        
        # If hand landmarks are found
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Extract keypoints
                keypoints = []
                for landmark in hand_landmarks.landmark:
                    keypoints.append([landmark.x, landmark.y, landmark.z])
                
                # Flatten the keypoints list and convert to numpy array
                keypoints = np.array(keypoints).flatten()
                
                # Ensure the input is compatible with the model
                keypoints = keypoints.reshape(1, -1)
                
                # Make a prediction
                prediction = model.predict(keypoints)
                predicted_class = np.argmax(prediction)
                predicted_label = labels[predicted_class]
                
                # Display the prediction on the frame
                cv2.putText(frame, f"Prediction: {predicted_label}", (10, 40), 
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                
                # Draw hand landmarks on the frame
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
        # Display the frame with prediction
        cv2.imshow("Real-Time Sign Language Detection", frame)
        
        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close all OpenCV windows
video_capture.release()
cv2.destroyAllWindows()
